In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import wave_functions as W
ws_path = '/gws/nopw/j04/ncas_climate_vol1/users/bc917929/'
wave_dir = f"{ws_path}wave_analysis/era5/"


In [2]:
Kelv_nc = xr.open_dataset(f'{wave_dir}uz_Kelvin_rm_clim_k2-40_p2-30_1980-2023_ERA5.nc').eastward_wind
Kelv_div = xr.open_dataarray(f'{wave_dir}div_Kelvin_rm_clim_k2-40_p2-30_1980-2023_ERA5.nc')
R1_nc = xr.open_dataset(f'{wave_dir}uvz_R1_rm_clim_kn2-40_p2-30_1980-2023_ERA5.nc')
WMRG_nc = xr.open_dataset(f'{wave_dir}uvz_WMRG_rm_clim_kn2-40_p2-30_1980-2023_ERA5.nc')

In [3]:
Kelv_nc,Kelv_div,R1_nc,WMRG_nc = [nc.sel(time = slice('2001','2018')) 
                                  for nc in [Kelv_nc,Kelv_div,R1_nc,WMRG_nc]]


In [4]:
Kelv_nc

<xarray.DataArray 'eastward_wind' (time: 6574, lon: 360, lat: 49)>
[115965360 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 24.0 23.0 22.0 21.0 20.0 ... -21.0 -22.0 -23.0 -24.0
  * lon      (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2018-12-31

In [5]:
Kelv_met = [Kelv_nc.sel(lat=0),Kelv_div.sel(lat=0)]
R1_met = [-1*R1_nc.eastward_wind.sel(lat=0),R1_nc.northward_wind.sel(lat=8)]
WMRG_met = [-1*WMRG_nc.eastward_wind.sel(lat=-10),WMRG_nc.northward_wind.sel(lat=0)]

wave_daily = dict(map(lambda nc,w: (w,nc.resample(time='1D').mean(['time'])),
        Kelv_met+R1_met+WMRG_met,['Kelv_x','Kelv_y','R1_x','R1_y','WMRG_x','WMRG_y']))

In [6]:
def rolling_nc_calc(nc):
    newnc = xr.concat([nc.isel(lon=slice(-2,None)).assign_coords({'lon':[-2,-1]}),
                nc,
                nc.isel(lon=slice(0,2)).assign_coords({'lon':[360,361]})],'lon')
    rolling_nc = newnc.rolling(lon=5,center=True).mean().isel(lon=slice(2,-2))
    return rolling_nc


In [7]:
wave_roll = {i:rolling_nc_calc(wave_daily[i]) for i in wave_daily}
wave_roll = {i[0]:i[1]/i[1].std('time') for i in wave_roll.items()}

In [8]:
wave_vals = [W.phase_func(wave_roll[f'{w}_x'],wave_roll[f'{w}_y'],w)
    for w in ['Kelv','R1','WMRG']]

In [9]:
wave_ncs = [xr.merge(i).expand_dims({'wave':[w]}) for i,w in zip(wave_vals,['Kelv','R1','WMRG'])]
wave_ncs = xr.concat(wave_ncs,'wave').drop('lat')
wave_ncs

<xarray.Dataset>
Dimensions:         (wave: 3, lon: 360, time: 6574)
Coordinates:
  * wave            (wave) object 'Kelv' 'R1' 'WMRG'
  * lon             (lon) float64 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * time            (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2018-12-31
Data variables:
    phi             (wave, time, lon) float64 3.457 3.731 3.905 ... 4.089 4.031
    wave_phase      (wave, time, lon) float32 2.0 2.0 2.0 3.0 ... 3.0 3.0 3.0
    wave_amplitude  (wave, time, lon) float64 0.302 0.3569 ... 0.8677 0.8723

In [10]:
wave_ncs.to_netcdf('./inputs/wave_phases_era5_2020-2023.nc')